# get_frequently_mutated Final 

In [23]:
import cptac
en = cptac.Endometrial()
#cptac.sync('endometrial', version='2.1')
#ov = cptac.Ovarian()
#cptac.sync(dataset='colon', version='0.0')
#cancer = cptac.Colon()

import pandas as pd
import numpy as np

CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but analysis may not be
published until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter cptac.embargo() to open the webpage for more details.


In [24]:
def get_frequently_mutated(cancer_type, cutoff = 0.1):  
    """
    take cancer object and find the frequently 
    mutated genes in the total tumors compared to the cutoff.

    Parameters:
    cancer_type (object): cancer class object from cptac module 
    cutoff (float): used as comparison to determine the 
                    status of gene mutation frequency

    Returns:
    freq_mutated_df (pd.DataFrame): DataFrame of frequently 
                mutated genes passing the cutoff. Columns 
                contain the fractions of total unique 
                mutations,missence type mutations, 
                and truncation type mutations per gene.
    
    The Missence_Mut column includes: 
        In_Frame_Del, In_Frame_Ins, Missense_Mutation
   
   The Truncation_Mut column includes: 
        Frame_Shift_Del, Frame_Shift_Ins, Splice_Site, 
        Nonsense_Mutation, Nonstop_Mutation
        
    These columns count multiple mutations of one gene in the 
    same sample, so fractions in the last two columns may 
    exceed the Unique_Samples_Mut column which only counts if 
    the gene was mutated once per sample."""    
    
    # get data frame
    somatic_mutations = cancer_type.get_mutations()
    sample_status_map = cancer_type._get_sample_status_map()
         
    '''Get total tumors/patients'''
    sample_status_series = sample_status_map.value_counts().reset_index()
    tumors = sample_status_series.loc[sample_status_series['index'] == 'Tumor']
    total_tumor_patients = tumors.iloc[0,1]
    print(total_tumor_patients)
        
    '''Find frequently mutated genes and their fraction of unique mutated samples.'''
    #move 'Sample_ID' from index to col 
    origin_df = somatic_mutations.reset_index()

    #group by gene and count unique samples
    genes_df = origin_df.groupby(['Gene']).nunique()

    #format
    genes_df = genes_df.rename(columns={"Sample_ID": "Unique_Samples_Mut"})
    genes_df = genes_df.drop(['Gene', 'Mutation','Location'], axis = 1)
    
    #filter using the cutoff and create fraction
    genes_df = genes_df.apply(lambda x: x / total_tumor_patients)
    filtered = genes_df.where(lambda x: x > cutoff)
    filtered_gene_df = filtered.dropna()
    
    '''Create Missence and Trucation data frame'''
    #create two categories in Mutation column
    '''
    if cancer_type.get_tumor_type() == 'Colon':
    
    missence_truncation_groups = {'frameshift substitution': 'T', 
        'frameshift deletion': 'T', 'frameshift insertion': 'T', 
        'stopgain': 'T ', 'stoploss':'T', 'nonsynonymous SNV': 'M',
        'nonframeshift insertion': 'M','nonframeshift deletion': 'M', 
        'nonframeshift substitution': 'M'}
        
    else: 
    '''
    missence_truncation_groups = {'In_Frame_Del': 'M', 'In_Frame_Ins': 'M',
        'Missense_Mutation': 'M', 'Frame_Shift_Del': 'T','Nonsense_Mutation': 'T', 
        'Splice_Site': 'T', 'Frame_Shift_Ins': 'T','Nonstop_Mutation':'T'}
       
    replaced_M_T = origin_df.replace(missence_truncation_groups)
    
    # group mutation categories
    miss = replaced_M_T.loc[replaced_M_T['Mutation'] == 'M']
    trunc = replaced_M_T.loc[replaced_M_T['Mutation'] == 'T']

    # group by gene and count unique samples for both categories
    miss_df = miss.groupby(['Gene']).nunique()
    trunc_df = trunc.groupby(['Gene']).nunique()

    #format
    miss_df = miss_df.rename(columns={"Sample_ID": "Missence_Mut"})
    miss_df = miss_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)

    trunc_df = trunc_df.rename(columns={"Sample_ID": "Truncation_Mut"})
    trunc_df = trunc_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)

    #join miss and trunc and change nan to 0, then divide by total tumors
    mut = miss_df.join(trunc_df).fillna(0)
    missence_and_truncation_df = mut.apply(lambda x: x / total_tumor_patients)


    '''Join data frames, keeping only the genes that passed the cutoff''' 
    freq_mutated_df = filtered_gene_df.join(missence_and_truncation_df).reset_index()
    freq_mutated_df.name = 'frequently_mutated'
                   
    return freq_mutated_df

Test Endometrial

In [53]:
df = get_frequently_mutated(en, .15)

95


In [54]:
df
#df.loc[df['Gene']]

,Gene,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
0,AHNAK,0.157895,0.126316,0.042105
1,ARID1A,0.452632,0.136842,0.400000
2,CCDC168,0.168421,0.115789,0.115789
3,CTCF,0.284211,0.094737,0.242105
4,CTNNB1,0.305263,0.305263,0.000000
5,DNAH17,0.157895,0.136842,0.031579
6,DOCK3,0.200000,0.115789,0.157895
7,FBXW7,0.189474,0.157895,0.042105
8,HERC2,0.178947,0.157895,0.063158
9,HUWE1,0.157895,0.157895,0.031579


In [69]:
#test results
m = en.get_mutations()
gene = 'INPPL1'
g = m.loc[m['Gene'] == gene]
print(g)
print(len(g.index.unique()))

r = g.groupby('Mutation')

r.groups


             Gene           Mutation       Location
Sample_ID                                          
S027       INPPL1        Splice_Site   p.X61_splice
S030       INPPL1    Frame_Shift_Del  p.R1156Gfs*46
S032       INPPL1    Frame_Shift_Del  p.R1156Gfs*46
S037       INPPL1    Frame_Shift_Del  p.P976Hfs*155
S037       INPPL1    Frame_Shift_Del  p.R1156Gfs*46
S046       INPPL1    Frame_Shift_Del   p.Q472Rfs*59
S046       INPPL1    Frame_Shift_Del  p.R1156Gfs*46
S049       INPPL1    Frame_Shift_Del  p.P976Hfs*155
S049       INPPL1    Frame_Shift_Del  p.R1156Gfs*46
S050       INPPL1    Frame_Shift_Del  p.R1156Gfs*46
S051       INPPL1    Frame_Shift_Del   p.S904Lfs*54
S058       INPPL1  Nonsense_Mutation        p.E230*
S059       INPPL1        Splice_Site  p.X500_splice
S069       INPPL1  Missense_Mutation       p.G1148C
S069       INPPL1    Frame_Shift_Del   p.R100Gfs*59
S073       INPPL1    Frame_Shift_Del  p.R1156Gfs*46
S077       INPPL1        Splice_Site  p.X680_splice
S078       I

{'Frame_Shift_Del': Index(['S030', 'S032', 'S037', 'S037', 'S046', 'S046', 'S049', 'S049', 'S050',
        'S051', 'S069', 'S073', 'S078', 'S078', 'S081', 'S085', 'S085', 'S088',
        'S091'],
       dtype='object', name='Sample_ID'),
 'Missense_Mutation': Index(['S069', 'S095'], dtype='object', name='Sample_ID'),
 'Nonsense_Mutation': Index(['S058'], dtype='object', name='Sample_ID'),
 'Splice_Site': Index(['S027', 'S059', 'S077'], dtype='object', name='Sample_ID')}

Test Colon

In [21]:
colon_freq_mutated_df = get_frequently_mutated(cancer, .25)

110


In [22]:
colon_freq_mutated_df

,Gene,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
0,APC,0.745455,0.100000,0.381818
1,CCDC168,0.263636,0.172727,0.145455
2,FAT3,0.254545,0.236364,0.009091
3,KRAS,0.318182,0.318182,0.000000
4,MUC16,0.354545,0.318182,0.054545
5,OBSCN,0.263636,0.236364,0.018182
6,SYNE1,0.263636,0.245455,0.054545
7,TP53,0.509091,0.345455,0.072727
8,TTN,0.545455,0.527273,0.136364


Test Ovarian

In [32]:
ovarian_freq_mutated_df = get_frequently_mutated(ov,.1)

111


In [33]:
ovarian_freq_mutated_df

,Gene,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
0,FCGBP,0.108108,0.063063,0.009009
1,MT-CO1,0.117117,0.090090,0.000000
2,MT-ND5,0.144144,0.063063,0.018018
3,MUC12,0.108108,0.108108,0.009009
4,MUC16,0.144144,0.108108,0.009009
5,MUC17,0.126126,0.108108,0.000000
6,MUC4,0.306306,0.234234,0.009009
7,MUC5B,0.108108,0.081081,0.000000
8,NCOR2,0.108108,0.027027,0.000000
9,TP53,0.693694,0.450450,0.243243
